# Data Importing and "Tidy" Data

### Needed packages

In [ ]:
library(tidyverse)
library(nycflights13)
library(fivethirtyeight)
# Packages needed internally
library(kableExtra)
library(knitr)
library(IRdisplay)
library(patchwork)

## Importing data

- Spreadsheet data is often saved in:
  - *Comma Separated Values* `.csv` file 
    * Each line in the file corresponds to one row of data/one observation
    * The values of different variables are separated by commas in each row
    * The first line is often, but not always, a *header* row indicating the names of the columns/variables
  - Excel `.xlsx` spreadsheet file
    - MicroSoft proprietary Excel software
    - `.xlsx` Excel files contain a lot of meta-data, include the use of bold and italic fonts, colored cells, different column widths, and formula macros
  - [Google Sheets](https://www.google.com/sheets/about/) file
    - A "cloud" or online-based way to work with a spreadsheet
    - Not discussed here

### Using the console

- Use the `read_csv()` function from `readr` (part of `tidyverse`) to read it off the web, import it into R, and save it in a data frame called `dem_score`

In [ ]:
(dem_score <- read_csv("https://moderndive.com/data/dem_score.csv"))

- `read_csv()` in `readr` is different from `read.csv()` in base-R
  - `read_csv()` saves data frames as `tibbles` by default
  - `read_csv()` is faster; overall is better  

## "Tidy" data

- A motivating example from the `fivethirtyeight` package
  - Look at the first 5 rows of the `drinks` data frame

In [ ]:
drinks %>%
    head(5)

- Tasks:
  1. `filter()` the `drinks` data frame to only consider 4 countries: the United States, China, Italy, and Saudi Arabia, *then*
  1. `select()` all columns except `total_litres_of_pure_alcohol` by using the `-` sign, *then*
  1. `rename()` the variables `beer_servings`, `spirit_servings`, and `wine_servings` to `beer`, `spirit`, and `wine`, respectively
  1. Save the resulting data frame in `drinks_smaller`

In [ ]:
drinks_smaller <- drinks %>%
    filter(country %in% c("USA", "China", "Italy", "Saudi Arabia")) %>% 
    select(-total_litres_of_pure_alcohol) %>% 
    rename(beer = beer_servings, 
           spirit = spirit_servings,
           wine = wine_servings)

- Create the side-by-side barplot

In [ ]:
# Will be explained later ...
drinks_smaller_tidy <- drinks_smaller %>% 
     pivot_longer(names_to = "type", 
                  values_to = "servings", 
                  cols = -country)
drink_smaller_tidy_plot <- ggplot(drinks_smaller_tidy,
    aes(x = country, y = servings, fill = type)) +
    geom_col(position = "dodge") +
    labs(x = "country", y = "servings")

1. The categorical variable `country` with four levels (China, Italy, Saudi Arabia, USA) would have to be mapped to the `x`-position of the bars
1. The numerical variable `servings` would have to be mapped to the `y`-position of the bars (the height of the bars)
1. The categorical variable `type` with three levels (beer, spirit, wine) would have to be mapped to the `fill` color of the bars

In [ ]:
drinks_smaller_tidy

- `drinks_smaller`: ["wide"](https://en.wikipedia.org/wiki/Wide_and_narrow_data) format
- `drinks_smaller_tidy` : ["long/narrow"](https://en.wikipedia.org/wiki/Wide_and_narrow_data#Narrow) format
  - THE "tidy" format: preferred in `tidyverse` 

### Definition of "tidy" data

> "Tidy" data is a standard way of mapping the meaning of a dataset to its structure; in *tidy data*:
>
> 1. Each variable forms a column
> 2. Each observation forms a row
> 3. Each type of observational unit forms a table

![Data Tidy Principle](./fig/tidy-1.png)

- Say you have the following table of stock prices:

In [ ]:
stocks <- tibble(
    Date = as.Date("2009-01-01") + 0:4,
    `Boeing stock price` = paste("$", c("173.55", "172.61", "173.86", "170.77", "174.29"), sep = ""),
    `Amazon stock price` = paste("$", c("174.90", "171.42", "171.58", "173.89", "170.16"), sep = ""),
    `Google stock price` = paste("$", c("174.34", "170.04", "173.65", "174.87", "172.19"), sep = "")) %>%
    slice(1:2)

stocks %>%
    kable(digits = 2,
          caption = "Stock prices (non-tidy format)",
          booktabs = TRUE,
          linesep = "") %>%
    kable_styling(font_size = 16) %>%
    as.character() %>%
    display_html()

- 2nd version

In [ ]:
stocks_tidy <- stocks %>%
    rename(Boeing = `Boeing stock price`,
           Amazon = `Amazon stock price`,
           Google = `Google stock price`) %>%
    pivot_longer(cols = -Date,
                 names_to = "Stock Name",
                 values_to = "Stock Price")
stocks_tidy %>%
      kable(digits = 2,
            caption = "Stock prices (tidy format)",
            booktabs = TRUE,
            linesep = "") %>%
      kable_styling(font_size = 16) %>%
      as.character() %>%
      display_html()

- 3rd version

In [ ]:
stocks <- tibble(Date = as.Date("2009-01-01") + 0:4,
    `Boeing Price` = paste("$", c("173.55", "172.61", "173.86", "170.77", "174.29"), sep = ""),
    `Weather` = c("Sunny", "Overcast", "Rain", "Rain", "Sunny")) %>%
     slice(1:2)

stocks %>%
    kable(digits = 2,
          caption = "Example of tidy data") %>%
    kable_styling(font_size = 16) %>%
    as.character() %>%
    display_html()

- The data *is* "tidy" since there are three variables corresponding to three unique pieces of information: Date, Boeing price, and the Weather that particular day

### Converting to "tidy" data

In [ ]:
drinks_smaller

- Convert it to "tidy" format by using `pivot_longer()` from `tidyverse` as follows:

In [ ]:
(drinks_smaller_tidy <- drinks_smaller %>% 
     pivot_longer(names_to = "type", 
                  values_to = "servings", 
                  cols = -country))

- Different code that produces the same output:

In [ ]:
drinks_smaller %>% 
    pivot_longer(names_to = "type", 
                 values_to = "servings", 
                 cols = c(beer, spirit, wine))

- Also for consecutive columns:

In [ ]:
drinks_smaller %>% 
    pivot_longer(names_to = "type", 
                 values_to = "servings", 
                 cols = beer:wine)

- Now we can produce the barplot: use `geom_col()` and not `geom_bar()` for we can map the "pre-counted" `servings` variable to the `y`-aesthetic of the bars

In [ ]:
ggplot(drinks_smaller_tidy,  
    aes(x = country, y = servings, fill = type)) +
    geom_col(position = "dodge")

- Tips:
  - Examples of `pivot_longer()` and `pivot_wider()` on the [tidyverse.org](https://tidyr.tidyverse.org/dev/articles/pivot.html#pew) webpage
    - Check out the different functions available for data tidying and a case study using data from the World Health Organization 
  - Example Data Sets: The R4DS Online Learning Community [TidyTuesday event](https://github.com/rfordatascience/tidytuesday) 

## Learning Check

Take a look the `airline_safety` data frame included in the `fivethirtyeight` data. Run the following:

In [ ]:
airline_safety

After reading the help file by running `?airline_safety`, we see that `airline_safety` is a data frame containing information on different airlines companies' safety records. This data was originally reported on the data journalism website FiveThirtyEight.com in Nate Silver's article ["Should Travelers Avoid Flying Airlines That Have Had Crashes in the Past?"](https://fivethirtyeight.com/features/should-travelers-avoid-flying-airlines-that-have-had-crashes-in-the-past/). Ignore the `incl_reg_subsidiaries` and `avail_seat_km_per_week` variables for simplicity:

In [ ]:
airline_safety_smaller <- airline_safety %>%
    select(-c(incl_reg_subsidiaries, avail_seat_km_per_week))
airline_safety_smaller

This data frame is not in "tidy" format. How would you convert this data frame to be in "tidy" format, in particular so that it has a variable `incident_type_years` indicating the incident type/year and a variable `count` of the counts?

**Solution**: 

This can been done using the `pivot_longer()` function from the `tidyr` package:

In [ ]:
airline_safety_smaller_tidy <- airline_safety_smaller %>%
    pivot_longer(names_to = "incident_type_years",
                 values_to = "count",
                 cols = -airline)
airline_safety_smaller_tidy

If you look at the resulting `airline_safety_smaller_tidy` data frame in the spreadsheet viewer, you'll see that the variable `incident_type_years` has 6 possible values: `"incidents_85_99", "fatal_accidents_85_99", "fatalities_85_99", 
"incidents_00_14", "fatal_accidents_00_14", "fatalities_00_14"` corresponding to the 6 columns of `airline_safety_smaller` we tidied. 

## Case study: Democracy in Guatemala

- Aim: produce a *time-series plot* showing how the democracy scores have changed over the 40 years from 1952 to 1992 for Guatemala
  - using `geom_line()`

In [ ]:
(guat_dem <- dem_score %>% 
    filter(country == "Guatemala"))

- Steps

  - Set `data = guat_dem` and use `geom_line()` layer 	
  - Need to map
    * `year` to the x-position aesthetic
    * `democracy_score` to the y-position aesthetic
    * years in `guat_dem` $\Rightarrow$ `year` , democracy score values in `guat_dem` $\Rightarrow$ `democracy_score` 
  - Resulting data frame with variables  `country`, `year`, `democracy_score`

In [ ]:
(guat_dem_tidy <- guat_dem %>% 
     pivot_longer(names_to = "year", 
                  values_to = "democracy_score", 
                  cols = -country,
                  names_transform = list(year = as.integer)))

1. `names_to` is the name of the variable in the new "tidy" data frame that will contain the *column names* of the original data
1. `values_to` is the name of the variable in the new "tidy" data frame that will contain the *values* of the original data
1. `cols` is the columns you either want to or don't want to "tidy" 
1. The last argument of `names_transform` tells R what type of variable `year` should be set to; `pivot_longer()` will set it to be a character value by default

- Create the time-series plot to visualize how democracy scores in Guatemala have changed from 1952 to 1992 
- Use the `labs()` function to add informative labels

In [ ]:
ggplot(guat_dem_tidy, 
       aes(x = year, y = democracy_score)) +
    geom_line() +
    labs(x = "Year", y = "Democracy Score")

- What if we forgot to include the `names_transform`?

In [ ]:
guat_dem_tidy <- guat_dem %>% 
     pivot_longer(names_to = "year", 
                  values_to = "democracy_score", 
                  cols = -country) 
ggplot(guat_dem_tidy, 
       aes(x = year, y = democracy_score)) +
    geom_line() +
    labs(x = "Year", y = "Democracy Score")

## Learning Check

Convert the `dem_score` data frame into a tidy data frame and assign the name of `dem_score_tidy` to the resulting long-formatted data frame.

**Solution**: Running the following in the console:

In [ ]:
dem_score_tidy <- dem_score %>%
  pivot_longer(names_to = "year", 
               values_to = "democracy_score",
               cols = -country)

Let's now compare the `dem_score` and `dem_score_tidy`. `dem_score` has democracy score information for each year in columns, whereas in `dem_score_tidy` there are explicit variables `year` and `democracy_score`. While both representations of the data contain the same information, we can only use `ggplot()` to create plots using the `dem_score_tidy` data frame.

In [ ]:
dem_score
dem_score_tidy

---

Read in the life expectancy data stored at <https://moderndive.com/data/le_mess.csv> and convert it to a tidy data frame. 

**Solution**:

In [ ]:
life_expectancy <- read_csv("https://moderndive.com/data/le_mess.csv")
life_expectancy_tidy <- life_expectancy %>%
    pivot_longer(names_to = "year",
                 values_to = "life_expectancy",
                 cols = -country)

We observe the same construct structure with respect to `year` in `life_expectancy` vs `life_expectancy_tidy` as we did in `dem_score` vs `dem_score_tidy`:

In [ ]:
life_expectancy
life_expectancy_tidy